In [1]:
import sys
#!{sys.executable} -m pip install librosa
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install scikit-learn
import numpy as np, keras, librosa, tensorflow, scipy

Using TensorFlow backend.


In [2]:
import os

In [8]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    if (max_pad_len>mfcc.shape[1]):
        pad_width = max_pad_len-mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)),mode='constant')
    else:
        mfcc = mfcc[:,:max_pad_len]
    return mfcc

In [9]:
import numpy as np
DATA_PATH = "./data/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, keras.utils.to_categorical(label_indices)

In [10]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels,_,_= get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        
        wavfiles = [path+label+'/'+wavfile for wavfile in os.listdir(path+'/'+label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
            
        np.save(label+'.npy',mfcc_vectors)

In [11]:
from sklearn.model_selection import train_test_split
def get_train_test(split_ratio=0.6, random_state=42):
    labels, indices,_ = get_labels(DATA_PATH)
    
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label+'.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
        
    assert X.shape[0] == len(y)
    
    return train_test_split(X, y, test_size=(1-split_ratio),random_state=random_state, shuffle=True)

In [12]:
save_data_to_array()

NoBackendError: 

In [13]:
X_train, X_test, y_train, y_test = get_train_test()
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = keras.utils.to_categorical(y_train)
y_test_hot = keras.utils.to_categorical(y_test)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu',input_shape=(20,11,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.fit(X_train, y_train_hot,batch_size=100, epochs=300, verbose=1, validation_data=(X_test, y_test_hot))


Train on 9261 samples, validate on 6175 samples
Epoch 1/300
9261/9261 [==============================] - 3s 288us/step - loss: 3.5733 - acc: 0.4117 - val_loss: 1.0163 - val_acc: 0.5823
Epoch 2/300
9261/9261 [==============================] - 2s 249us/step - loss: 1.0569 - acc: 0.5476 - val_loss: 0.8251 - val_acc: 0.6447
Epoch 3/300
9261/9261 [==============================] - 2s 248us/step - loss: 0.8964 - acc: 0.6266 - val_loss: 0.6735 - val_acc: 0.7495
Epoch 4/300
9261/9261 [==============================] - 2s 245us/step - loss: 0.8038 - acc: 0.6671 - val_loss: 0.6268 - val_acc: 0.7401
Epoch 5/300
9261/9261 [==============================] - 2s 251us/step - loss: 0.7251 - acc: 0.7083 - val_loss: 0.5970 - val_acc: 0.7817
Epoch 6/300
9261/9261 [==============================] - 3s 280us/step - loss: 0.6854 - acc: 0.7285 - val_loss: 0.5777 - val_acc: 0.7760
Epoch 7/300
9261/9261 [==============================] - 2s 263us/step - loss: 0.6241 - acc: 0.7539 - val_loss: 0.5308 - val_acc: 

9261/9261 [==============================] - 2s 240us/step - loss: 0.2134 - acc: 0.9203 - val_loss: 0.3173 - val_acc: 0.8946
Epoch 61/300
9261/9261 [==============================] - 2s 236us/step - loss: 0.2099 - acc: 0.9218 - val_loss: 0.3072 - val_acc: 0.8968
Epoch 62/300
9261/9261 [==============================] - 2s 239us/step - loss: 0.2044 - acc: 0.9246 - val_loss: 0.3099 - val_acc: 0.8962
Epoch 63/300
9261/9261 [==============================] - 2s 240us/step - loss: 0.2097 - acc: 0.9209 - val_loss: 0.3027 - val_acc: 0.9009
Epoch 64/300
9261/9261 [==============================] - 2s 237us/step - loss: 0.2024 - acc: 0.9234 - val_loss: 0.3048 - val_acc: 0.8918
Epoch 65/300
9261/9261 [==============================] - 2s 241us/step - loss: 0.1925 - acc: 0.9264 - val_loss: 0.3083 - val_acc: 0.8905
Epoch 66/300
9261/9261 [==============================] - 2s 247us/step - loss: 0.1975 - acc: 0.9280 - val_loss: 0.3192 - val_acc: 0.8946
Epoch 67/300
9261/9261 [=======================

9261/9261 [==============================] - 2s 238us/step - loss: 0.1425 - acc: 0.9485 - val_loss: 0.3502 - val_acc: 0.8970
Epoch 120/300
9261/9261 [==============================] - 2s 239us/step - loss: 0.1461 - acc: 0.9461 - val_loss: 0.3268 - val_acc: 0.9004
Epoch 121/300
9261/9261 [==============================] - 2s 235us/step - loss: 0.1306 - acc: 0.9514 - val_loss: 0.3400 - val_acc: 0.8977
Epoch 122/300
9261/9261 [==============================] - 2s 240us/step - loss: 0.1391 - acc: 0.9501 - val_loss: 0.3641 - val_acc: 0.8993
Epoch 123/300
9261/9261 [==============================] - 2s 237us/step - loss: 0.1426 - acc: 0.9481 - val_loss: 0.4240 - val_acc: 0.8821
Epoch 124/300
9261/9261 [==============================] - 2s 237us/step - loss: 0.1336 - acc: 0.9509 - val_loss: 0.3720 - val_acc: 0.8985
Epoch 125/300
9261/9261 [==============================] - 2s 241us/step - loss: 0.1449 - acc: 0.9472 - val_loss: 0.3201 - val_acc: 0.9012
Epoch 126/300
9261/9261 [================

9261/9261 [==============================] - 2s 235us/step - loss: 0.1159 - acc: 0.9563 - val_loss: 0.3554 - val_acc: 0.9035
Epoch 179/300
9261/9261 [==============================] - 2s 237us/step - loss: 0.1152 - acc: 0.9621 - val_loss: 0.3447 - val_acc: 0.8989
Epoch 180/300
9261/9261 [==============================] - 2s 238us/step - loss: 0.1126 - acc: 0.9596 - val_loss: 0.3771 - val_acc: 0.8996
Epoch 181/300
9261/9261 [==============================] - 2s 239us/step - loss: 0.1186 - acc: 0.9582 - val_loss: 0.3404 - val_acc: 0.8983
Epoch 182/300
9261/9261 [==============================] - 2s 237us/step - loss: 0.1101 - acc: 0.9644 - val_loss: 0.3726 - val_acc: 0.9017
Epoch 183/300
9261/9261 [==============================] - 2s 257us/step - loss: 0.1168 - acc: 0.9594 - val_loss: 0.3539 - val_acc: 0.9015
Epoch 184/300
9261/9261 [==============================] - 3s 328us/step - loss: 0.1131 - acc: 0.9592 - val_loss: 0.3672 - val_acc: 0.8985
Epoch 185/300
9261/9261 [================

9261/9261 [==============================] - 4s 434us/step - loss: 0.0958 - acc: 0.9652 - val_loss: 0.3940 - val_acc: 0.9001
Epoch 238/300
9261/9261 [==============================] - 3s 366us/step - loss: 0.1039 - acc: 0.9624 - val_loss: 0.3923 - val_acc: 0.9027
Epoch 239/300
9261/9261 [==============================] - 3s 347us/step - loss: 0.1024 - acc: 0.9624 - val_loss: 0.3560 - val_acc: 0.9032
Epoch 240/300
9261/9261 [==============================] - 3s 325us/step - loss: 0.1109 - acc: 0.9600 - val_loss: 0.3653 - val_acc: 0.9043
Epoch 241/300
9261/9261 [==============================] - 3s 320us/step - loss: 0.1128 - acc: 0.9618 - val_loss: 0.3846 - val_acc: 0.8994
Epoch 242/300
9261/9261 [==============================] - 3s 325us/step - loss: 0.1007 - acc: 0.9618 - val_loss: 0.3953 - val_acc: 0.8975
Epoch 243/300
9261/9261 [==============================] - 3s 324us/step - loss: 0.0985 - acc: 0.9636 - val_loss: 0.3865 - val_acc: 0.9049
Epoch 244/300
9261/9261 [================

9261/9261 [==============================] - 3s 323us/step - loss: 0.0864 - acc: 0.9713 - val_loss: 0.3859 - val_acc: 0.9033
Epoch 297/300
9261/9261 [==============================] - 3s 353us/step - loss: 0.0956 - acc: 0.9687 - val_loss: 0.3834 - val_acc: 0.9023
Epoch 298/300
9261/9261 [==============================] - 3s 294us/step - loss: 0.0787 - acc: 0.9735 - val_loss: 0.4170 - val_acc: 0.9012
Epoch 299/300
9261/9261 [==============================] - 2s 265us/step - loss: 0.0954 - acc: 0.9673 - val_loss: 0.4121 - val_acc: 0.8993
Epoch 300/300
9261/9261 [==============================] - 2s 259us/step - loss: 0.0883 - acc: 0.9697 - val_loss: 0.4038 - val_acc: 0.9022


In [15]:
sample = wav2mfcc('./GoRecording2.wav')
sample_reshaped = sample.reshape(1,20,11,1)
print(get_labels()[0][np.argmax(model.predict(sample_reshaped))])

go


In [22]:
from pydub import AudioSegment

/usr/local/lib/python3.6/dist-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [25]:
import pyaudio
import wave
import time
import sys
import pygame

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [30]:
import socket
import numpy as np
from pygame import mixer
'''
try:
        mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        print ("Socket created")

except socket.error:
        print ("Socket connection failed")

port = 8888
try:
        mySock.connect(("192.168.43.195", port))
        print ("Connected to RPi!")
except socket.error:
        print ("Not able to connect")
'''
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "out.wav"
while True:
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)

        print("* recording")

        frames = []

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

        print("* done recording")

        stream.stop_stream()
        stream.close()
        p.terminate()

        wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        
        recording = AudioSegment.from_wav("./out.wav")
        recording+=20
        recording.export("./out.wav", format="wav")

        print("Before the sleep statement")
        mixer.init(frequency=44100)
        mixer.music.load("./out.wav")
        mixer.music.play()
        time.sleep(5)
        print("After the sleep statement")

        sample = wav2mfcc('./out.wav')
        sample_reshaped = sample.reshape(1,20,11,1)
        message = get_labels()[0][np.argmax(model.predict(sample_reshaped))]       
        print (message)
        #while message.lower().strip!='6':
        '''
        mySock.send(message.encode())
        data = mySock.recv(1024).decode()
        print ("Received from server: " + data)
                #message = input("->")

mySock.close()
'''

* recording
* done recording
Before the sleep statement
After the sleep statement
stop
* recording
* done recording
Before the sleep statement
After the sleep statement
right
* recording
* done recording
Before the sleep statement
After the sleep statement
stop
* recording
* done recording
Before the sleep statement


KeyboardInterrupt: 